In [ ]:
#16.9.1 PySpark ETL
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://baylex-s3-example-bucket.s3.amazonaws.com/user_data.csv"
# url ="https://YOUR-BUCKET-NAME.s3.amazonaws.com/user_data.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
# Show DataFrame
user_data_df.show()

In [ ]:
# add in the other csv
url = "https://baylex-s3-example-bucket.s3.amazonaws.com/user_payment.csv"
# url ="https://YOUR-BUCKET-NAME.s3.amazonaws.com/user_payment.csv"
spark.sparkContext.addFile(url)
user_payment_df = spark.read.csv(SparkFiles.get("user_payment.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
user_payment_df.show()

In [ ]:
# Join the two DataFrame
joined_df = user_data_df.join(user_payment_df,on="username",how="inner")
joined_df.show()

In [ ]:
# Drop null values
dropna_df = joined_df.dropna()
dropna_df.show()

In [ ]:
# Load is a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
cleaned_df = dropna_df.filter(col("active_user") == True)
cleaned_df.show()

In [ ]:
# Create user dataframe to match active_user table
clean_user_df = cleaned_df.select(["id", "first_name","last_name","username"])
cleaned_df.show()

In [ ]:
# Create user dataframe to match billing_info table
clean_billing_df = cleaned_df.select(["billing_id","street_address","state","username"])
clean_billing_df.show()

In [ ]:
# Create user dataframe to match payment_info table
clean_payment_df = cleaned_df.select(["billing_id","cc_encrypted"])
clean_payment_df.show()

In [ ]:
# Configure settings for RDS
mode = "append"
# jdbc_url="jdbc:postgresql://<connection string in AWS>:5432/<database-name in pgAdmin>"
# amazon-reviews.camjftkrswax.us-east-1.rds.amazonaws.com
jdbc_url="jdbc:postgresql://dataviz.camjftkrswax.us-east-1.rds.amazonaws.com:5432/postgres"
#
config = {"user":"postgres",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to active_user table in RDS
clean_user_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)

In [ ]:
# Write dataframe to billing_info table in RDS
clean_billing_df.write.jdbc(url=jdbc_url, table='billing_info', mode=mode, properties=config)

In [ ]:
# Write dataframe to payment_info table in RDS
clean_payment_df.write.jdbc(url=jdbc_url, table='payment_info', mode=mode, properties=config)

In [ ]:
# inside of pgAdmin
# -- Query database to check successful upload
# SELECT * FROM active_user;
# SELECT * FROM billing_info;
# SELECT * FROM payment_info;